In [1]:
#  For reproducing results
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
# Might help if the notebook kernel dies
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Подготовка данных

In [4]:
train_set = pd.read_csv('fashion-mnist_train.csv')
test_set = pd.read_csv('fashion-mnist_test.csv')

In [5]:
X_train = train_set.loc[:, train_set.columns != 'label'] 
X_test = test_set.loc[:, test_set.columns != 'label'] 

In [6]:
y_train = train_set['label']
y_test = test_set['label']

In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [8]:
# Normalizing data
X_train /= 255
X_test /= 255

In [9]:
# Transforming labels into one-hot vectors
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.1)

In [11]:
batch_size = 512
epochs=20

# Logistic regression

In [12]:
model_name = 'log_regression'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

In [13]:
lr_model = tf.keras.models.Sequential()
lr_model.add(tf.keras.layers.Dense(num_classes, activation='softmax', input_shape=(784,)))

In [14]:
lr_model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

In [15]:
lr_model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 2s 36us/step - loss: 1.6539 - acc: 0.5159 - val_loss: 1.2969 - val_acc: 0.6538
Epoch 2/20
54000/54000 [==============================] - 1s 27us/step - loss: 1.1670 - acc: 0.6685 - val_loss: 1.0514 - val_acc: 0.6922
Epoch 3/20
54000/54000 [==============================] - 1s 26us/step - loss: 1.0013 - acc: 0.6943 - val_loss: 0.9376 - val_acc: 0.7123
Epoch 4/20
54000/54000 [==============================] - 1s 26us/step - loss: 0.9136 - acc: 0.7129 - val_loss: 0.8702 - val_acc: 0.7275
Epoch 5/20
54000/54000 [==============================] - 1s 26us/step - loss: 0.8569 - acc: 0.7282 - val_loss: 0.8217 - val_acc: 0.7440
Epoch 6/20
54000/54000 [==============================] - 1s 26us/step - loss: 0.8160 - acc: 0.7411 - val_loss: 0.7863 - val_acc: 0.7525
Epoch 7/20
54000/54000 [==============================] - 1s 27us/step - loss: 0.7847 - acc: 0.7503 - val_loss: 0.7592 - val_acc:

In [16]:
loss, accuracy = lr_model.evaluate(X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.6085037322044372
Test accuracy: 0.8048333333333333


# Fully Connected Neural Network

In [17]:
# Model from the 5.2.6 lecture but without drop-out
model_name = 'NN_1'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

NN_model = tf.keras.models.Sequential()
NN_model.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)))
NN_model.add(tf.keras.layers.Dense(512, activation='relu'))
NN_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [18]:
NN_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

NN_model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 7s 129us/step - loss: 0.5923 - acc: 0.7939 - val_loss: 0.3991 - val_acc: 0.8607
Epoch 2/20
54000/54000 [==============================] - 6s 113us/step - loss: 0.3926 - acc: 0.8600 - val_loss: 0.3592 - val_acc: 0.8703
Epoch 3/20
54000/54000 [==============================] - 6s 116us/step - loss: 0.3455 - acc: 0.8750 - val_loss: 0.3313 - val_acc: 0.8830
Epoch 4/20
54000/54000 [==============================] - 6s 114us/step - loss: 0.3117 - acc: 0.8866 - val_loss: 0.3215 - val_acc: 0.8825
Epoch 5/20
54000/54000 [==============================] - 6s 119us/step - loss: 0.2935 - acc: 0.8923 - val_loss: 0.3162 - val_acc: 0.8777
Epoch 6/20
54000/54000 [==============================] - 7s 121us/step - loss: 0.2794 - acc: 0.8988 - val_loss: 0.3083 - val_acc: 0.8902
Epoch 7/20
54000/54000 [==============================] - 6s 119us/step - loss: 0.2622 - acc: 0.9033 - val_loss: 0.2990 - v

In [19]:
loss, accuracy = NN_model.evaluate(X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.35368617677688596
Test accuracy: 0.8826666666666667


In [20]:
# Adding drop-out
model_name = 'NN_2'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

NN_model2 = tf.keras.models.Sequential()
NN_model2.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)))
NN_model2.add(tf.keras.layers.Dropout(0.2))
NN_model2.add(tf.keras.layers.Dense(512, activation='relu'))
NN_model2.add(tf.keras.layers.Dropout(0.2))
NN_model2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [21]:
NN_model2.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

NN_model2.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 10s 191us/step - loss: 0.6364 - acc: 0.7731 - val_loss: 0.4157 - val_acc: 0.8543
Epoch 2/20
54000/54000 [==============================] - 9s 173us/step - loss: 0.4132 - acc: 0.8511 - val_loss: 0.3660 - val_acc: 0.8650
Epoch 3/20
54000/54000 [==============================] - 9s 174us/step - loss: 0.3707 - acc: 0.8653 - val_loss: 0.3362 - val_acc: 0.8772
Epoch 4/20
54000/54000 [==============================] - 9s 173us/step - loss: 0.3435 - acc: 0.8747 - val_loss: 0.3359 - val_acc: 0.8755
Epoch 5/20
54000/54000 [==============================] - 9s 173us/step - loss: 0.3267 - acc: 0.8800 - val_loss: 0.3186 - val_acc: 0.8822
Epoch 6/20
54000/54000 [==============================] - 9s 171us/step - loss: 0.3047 - acc: 0.8880 - val_loss: 0.3057 - val_acc: 0.8875
Epoch 7/20
54000/54000 [==============================] - 9s 173us/step - loss: 0.2951 - acc: 0.8917 - val_loss: 0.2976 - 

In [22]:
loss, accuracy = NN_model2.evaluate(X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.2798681590259075
Test accuracy: 0.898


### Вывод

Полносвязная нейросеть работает лучше, чем логистическая регрессия, т.к. это более сложная модель, которая позволяет обнаружить больше зависимостей в данных.

# Convolutional Neural Network

In [23]:
CNN_X_train = X_train.as_matrix()
CNN_X_val = X_val.as_matrix()
CNN_X_test = X_test.as_matrix()

CNN_X_train = CNN_X_train.reshape(X_train.shape[0], 28, 28, 1)
CNN_X_val = CNN_X_val.reshape(X_val.shape[0], 28, 28, 1)
CNN_X_test = CNN_X_test.reshape(X_test.shape[0], 28, 28, 1)

/Users/itrechyokas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/itrechyokas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/itrechyokas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# Model from task 5.2
model_name = 'CNN_1'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model = tf.keras.models.Sequential()
CNN_model.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
CNN_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu'))
CNN_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model.add(tf.keras.layers.Flatten())
CNN_model.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [25]:
CNN_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 35s 657us/step - loss: 0.8442 - acc: 0.7214 - val_loss: 0.5043 - val_acc: 0.8160
Epoch 2/20
54000/54000 [==============================] - 35s 643us/step - loss: 0.4642 - acc: 0.8323 - val_loss: 0.4011 - val_acc: 0.8600
Epoch 3/20
54000/54000 [==============================] - 34s 635us/step - loss: 0.3973 - acc: 0.8596 - val_loss: 0.3863 - val_acc: 0.8623
Epoch 4/20
54000/54000 [==============================] - 34s 633us/step - loss: 0.3618 - acc: 0.8705 - val_loss: 0.3499 - val_acc: 0.8767
Epoch 5/20
54000/54000 [==============================] - 34s 633us/step - loss: 0.3369 - acc: 0.8814 - val_loss: 0.3311 - val_acc: 0.8857
Epoch 6/20
54000/54000 [==============================] - 34s 633us/step - loss: 0.3165 - acc: 0.8877 - val_loss: 0.3258 - val_acc: 0.8835
Epoch 7/20
54000/54000 [==============================] - 35s 640us/step - loss: 0.3032 - acc: 0.8931 - val_loss: 0.3

In [26]:
loss, accuracy = CNN_model.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.25830628256003063
Test accuracy: 0.905


In [27]:
# Adding 2 more layers (also added padding as without it I got 'Negative dimension size...' error)
model_name = 'CNN_2'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model2 = tf.keras.models.Sequential()
CNN_model2.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model2.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model2.add(tf.keras.layers.Flatten())
CNN_model2.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [28]:
CNN_model2.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model2.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 56s 1ms/step - loss: 0.9426 - acc: 0.6607 - val_loss: 0.5647 - val_acc: 0.7855
Epoch 2/20
54000/54000 [==============================] - 55s 1ms/step - loss: 0.5137 - acc: 0.8132 - val_loss: 0.4300 - val_acc: 0.8422
Epoch 3/20
54000/54000 [==============================] - 55s 1ms/step - loss: 0.4216 - acc: 0.8475 - val_loss: 0.3975 - val_acc: 0.8507
Epoch 4/20
54000/54000 [==============================] - 55s 1ms/step - loss: 0.3739 - acc: 0.8649 - val_loss: 0.3596 - val_acc: 0.8653
Epoch 5/20
54000/54000 [==============================] - 55s 1ms/step - loss: 0.3415 - acc: 0.8743 - val_loss: 0.3185 - val_acc: 0.8882
Epoch 6/20
54000/54000 [==============================] - 54s 1ms/step - loss: 0.3302 - acc: 0.8785 - val_loss: 0.3234 - val_acc: 0.8890
Epoch 7/20
54000/54000 [==============================] - 54s 1ms/step - loss: 0.3081 - acc: 0.8882 - val_loss: 0.2998 - val_acc:

In [29]:
loss, accuracy = CNN_model2.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.24838692478338878
Test accuracy: 0.9138333333333334


In [30]:
# Adding batch mnormalization after each convolution layer
model_name = 'CNN_3'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model3 = tf.keras.models.Sequential()
CNN_model3.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model3.add(tf.keras.layers.BatchNormalization())
CNN_model3.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model3.add(tf.keras.layers.Flatten())
CNN_model3.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model3.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [31]:
CNN_model3.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model3.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 106s 2ms/step - loss: 0.5279 - acc: 0.8163 - val_loss: 2.2723 - val_acc: 0.1517
Epoch 2/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.2904 - acc: 0.8956 - val_loss: 2.0966 - val_acc: 0.3148
Epoch 3/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.2443 - acc: 0.9115 - val_loss: 1.6132 - val_acc: 0.5293
Epoch 4/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.2126 - acc: 0.9233 - val_loss: 0.9873 - val_acc: 0.6653
Epoch 5/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.1821 - acc: 0.9335 - val_loss: 0.3595 - val_acc: 0.8772
Epoch 6/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.1621 - acc: 0.9405 - val_loss: 0.2829 - val_acc: 0.8972
Epoch 7/20
54000/54000 [==============================] - 103s 2ms/step - loss: 0.1442 - acc: 0.9484 - val_loss: 0.2404 - v

In [42]:
loss, accuracy = CNN_model3.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.4267293870647748
Test accuracy: 0.9036666666666666


In [33]:
# Adding drop-out after each max pooling layer
model_name = 'CNN_4'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model4 = tf.keras.models.Sequential()
CNN_model4.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model4.add(tf.keras.layers.BatchNormalization())
CNN_model4.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
CNN_model4.add(tf.keras.layers.Dropout(0.2))

CNN_model4.add(tf.keras.layers.Flatten())
CNN_model4.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model4.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [34]:
CNN_model4.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model4.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 141s 3ms/step - loss: 0.8237 - acc: 0.7002 - val_loss: 2.2467 - val_acc: 0.0965
Epoch 2/20
54000/54000 [==============================] - 138s 3ms/step - loss: 0.4730 - acc: 0.8271 - val_loss: 2.6231 - val_acc: 0.1237
Epoch 3/20
54000/54000 [==============================] - 138s 3ms/step - loss: 0.3950 - acc: 0.8572 - val_loss: 2.0638 - val_acc: 0.3333
Epoch 4/20
54000/54000 [==============================] - 138s 3ms/step - loss: 0.3586 - acc: 0.8706 - val_loss: 0.9502 - val_acc: 0.6933
Epoch 5/20
54000/54000 [==============================] - 138s 3ms/step - loss: 0.3299 - acc: 0.8807 - val_loss: 0.4386 - val_acc: 0.8555
Epoch 6/20
54000/54000 [==============================] - 138s 3ms/step - loss: 0.3070 - acc: 0.8890 - val_loss: 0.2838 - val_acc: 0.8937
Epoch 7/20
54000/54000 [==============================] - 137s 3ms/step - loss: 0.2950 - acc: 0.8932 - val_loss: 0.2447 - v

In [41]:
loss, accuracy = CNN_model4.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.20755504260460536
Test accuracy: 0.921


In [37]:
# Tried batch normalization after fully connected layer
model_name = 'CNN_5'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/{model_name}')

CNN_model5 = tf.keras.models.Sequential()
CNN_model5.add(tf.keras.layers.Convolution2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(28,28,1)))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Convolution2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', ))
CNN_model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

CNN_model5.add(tf.keras.layers.Flatten())
CNN_model5.add(tf.keras.layers.Dense(64, activation='relu'))
CNN_model5.add(tf.keras.layers.BatchNormalization())
CNN_model5.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

In [38]:
CNN_model5.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

CNN_model5.fit(
    CNN_X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(CNN_X_val , y_val),
    callbacks=[tensorboard]
)

Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 59s 1ms/step - loss: 0.6407 - acc: 0.7918 - val_loss: 1.4976 - val_acc: 0.7563
Epoch 2/20
54000/54000 [==============================] - 58s 1ms/step - loss: 0.3477 - acc: 0.8789 - val_loss: 0.9252 - val_acc: 0.8737
Epoch 3/20
54000/54000 [==============================] - 58s 1ms/step - loss: 0.2888 - acc: 0.8978 - val_loss: 0.5265 - val_acc: 0.8818
Epoch 4/20
54000/54000 [==============================] - 57s 1ms/step - loss: 0.2559 - acc: 0.9093 - val_loss: 0.3377 - val_acc: 0.8933
Epoch 5/20
54000/54000 [==============================] - 57s 1ms/step - loss: 0.2297 - acc: 0.9181 - val_loss: 0.2698 - val_acc: 0.9122
Epoch 6/20
54000/54000 [==============================] - 56s 1ms/step - loss: 0.2099 - acc: 0.9245 - val_loss: 0.2891 - val_acc: 0.8978
Epoch 7/20
54000/54000 [==============================] - 57s 1ms/step - loss: 0.1951 - acc: 0.9300 - val_loss: 0.2323 - val_acc:

In [39]:
loss, accuracy = CNN_model5.evaluate(CNN_X_val, y_val, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.4377724414467812
Test accuracy: 0.8905


### Вывод

Сверточная нейронная сеть не дает большого прироста качества по сравнению с полносвязной нейросетью (всего 0.01 пункт accuracy). Увеличение слоев и добавление batch normalization также дают совсем небольшой прирост качества.  

# Выбор лучшей модели

Судя по значению метрики accuracy на валидационных данных, наилучший результат имеет модель CNN_4 (уже на 9 эпохе обучения) - 0.92. Посмотрим, как будет вести себя эта модель на новых данных, посчитав метрику accuracy на тестовом сете.

In [40]:
loss, accuracy = CNN_model4.evaluate(CNN_X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.21060755120813845
Test accuracy: 0.9214
